https://python.langchain.com/v0.1/docs/get_started/introduction/

!pip install python-dotenv
!pip install langchain
!pip install langchain-openai

In [ ]:
import openai
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

In [ ]:
# Se cargan los LLMs

from langchain_openai  import ChatOpenAI # Para chat models
from langchain_openai import OpenAI # Para completion models

In [ ]:
llm = ChatOpenAI()
output = llm.invoke("Cual es la capital de Argentina") # En este caso se usa un modelo conversacional
print(output.content)

# 1- PROMPT + TEMPLATES

## A- PromptTemplate

* `PromptTemplate` consta de un `template` en formato de `str`.
* Acepta un conjunto de parámetros del usuario que se pueden utilizar para generar un mensaje para un `LLM`.
* El `template` se puede formatear usando `f-strings`.

In [ ]:
from langchain.prompts import PromptTemplate

**Usando un modelo conversacional:**

In [ ]:
template = """
Tell me a joke about {topic}, make it funny
and in {language}
"""
print(f"template:{template}")

prompt_template = PromptTemplate.from_template(template = template) # En este ejemplo, la variable 'template' es un 'str'.
print(f"prompt_template:\n{prompt_template}")

print()

prompt = prompt_template.format(topic = "programation", language = "german")
print(f"prompt:{prompt}")

In [ ]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo")
output = llm.invoke(prompt)
print(output.content)

In [ ]:
# Se repite el ejemplo, pero sin variables

template = """
Tell me a joke
"""
print(f"template:{template}")

prompt_template = PromptTemplate.from_template(template = template)
print(f"prompt_template:\n{prompt_template}")

print()

prompt = prompt_template.format() # En este caso no pasamos ninguna variable
print(f"prompt:{prompt}")

In [ ]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo")
output = llm.invoke(prompt)
print(output.content)

**Usando un modelo de completion:**

In [ ]:
template = """
Escribí una función llamada '{nombre_funcion}' en código {language}
que ejecute la siguiente:
###
{tarea}
###
"""
print(f"template:{template}")

prompt_template = PromptTemplate.from_template(template = template)
print(f"prompt_template:\n{prompt_template}")

print()

prompt = prompt_template.format(nombre_funcion = "imprimir_numeros",
                                language = "python",
                                tarea = "Imprimí los números del 1 al 10")
print(f"prompt:{prompt}")

In [ ]:
llm = OpenAI(model_name = "gpt-3.5-turbo-instruct", temperature = 0)

output = llm.invoke(prompt)
print(output)

## B- ChatPromptTemplate

* `ChatPromptTemplate` se compone de una lista de mensajes.
* Cada elemento nuevo es un mensaje nuevo en el mensaje final.
* Cada mensaje tiene tiene dos parámetros: `role` y `content`.
* `Message`: cuando no hay variables para formatear.
* `MessageTemplate`: cuando hay variables para formatear.

**Usando únicamente ChatPromptTemplate:**

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
# Notar que estoy definiendo de forma manual si el mensaje es de tipo:
    # 'system'
    # 'human'
    # 'ai'

chat_template = ChatPromptTemplate.from_messages(messages =
    [
        ("system", "You are a helpful AI bot. Your name is {name}."), # Hay variables para formatear
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)
print(f"chat_template:")
for i in chat_template:
    print(i)
    print("")

In [ ]:
chat_template.messages

In [ ]:
messages = chat_template.format_messages(name = "Bob", user_input = "What is your name?")
messages

In [ ]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo")
output = llm.invoke(messages)
print(output.content)

**Usando únicamente 'Message' (no hay variables para formatear):**

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [ ]:
chat_template = ChatPromptTemplate.from_messages(messages =
    [
        SystemMessage(content = "You are a helpful AI bot. Your name is Bob."), # Pasamos la variable de forma manual
        HumanMessage(content = "Hello, how are you doing?"),
        AIMessage(content = "I'm doing well, thanks!"),
        HumanMessage(content = "What is your name?"), # Pasamos la variable de forma manual
    ]
)
print(f"chat_template:")
for i in chat_template:
    print(i)

In [ ]:
messages = chat_template.format_messages()
messages

In [ ]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo")
output = llm.invoke(messages)
print(output.content)

**Usando únicamente 'MessagePromptTemplate' (hay variables para formatear):**

In [ ]:
from langchain_core.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate

In [ ]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(messages =
    [
        SystemMessagePromptTemplate.from_template(template = "You are a helpful AI bot. Your name is {name}."),
        HumanMessagePromptTemplate.from_template(template = "{user_input}")
    ]
)

print(f"chat_template:")
for i in chat_template:
    print(i)

In [ ]:
messages = chat_template.format_messages(name = "Bob", user_input = "What is your name?")
messages

In [ ]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo")
output = llm.invoke(messages)
print(output.content)

**Combinando 'Message' con 'MessagePromptTemplate':**

In [ ]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

In [ ]:
chat_template = ChatPromptTemplate.from_messages(messages =
    [
        SystemMessagePromptTemplate.from_template(template = "You are a helpful AI bot. Your name is {name}."), # Como hay variables para formatear se utiliza 'MessagePromptTemplate'
        HumanMessage(content = "Hello, how are you doing?"), # Como no hay variables para formatear se utiliza 'Message'
        AIMessage(content = "I'm doing well, thanks!"), # Como no hay variables para formatear se utiliza 'Message'
        HumanMessagePromptTemplate.from_template(template = "{user_input}"), # Como hay variables para formatear se utiliza 'MessagePromptTemplate'
    ]
)
print(f"chat_template:")
for i in chat_template:
    print(i)

In [ ]:
messages = chat_template.format_messages(name = "Bob", user_input = "What is your name?")
messages

In [ ]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo")
output = llm.invoke(messages)
print(output.content)

**Otro ejemplo combinando 'Message' con 'MessagePromptTemplate':**

In [ ]:
chat_template = ChatPromptTemplate.from_messages(messages =
    [
        SystemMessage(content = "Responde con formato json."), # Como no hay variables para formatear se utiliza 'Message'
        HumanMessagePromptTemplate.from_template(template = "Nombra los {n} países de {area} con mayor cantidad de habitantes.") # Como hay variables para formatear se utiliza 'MessagePromptTemplate'
    ]
)

In [ ]:
messages = chat_template.format_messages(n = "10", area = "Europe")
messages

In [ ]:
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

In [ ]:
output

**¿Cómo podemos definir el formato 'json' como una variable a pasar?:**

In [ ]:
chat_template = ChatPromptTemplate.from_messages(messages =
    [
        SystemMessagePromptTemplate.from_template(template = "Responde con formato {formato}."),
        HumanMessagePromptTemplate.from_template(template = "Nombra los {n} países de {area} con mayor cantidad de habitantes.")
    ]
)

In [ ]:
messages = chat_template.format_messages(formato = "json", n = "10", area = "Europe")
messages

In [ ]:
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

# 2- TÉCNICAS

## A- Zero-Shot

In [ ]:
# Recordar la estructura template --> prompt_template --> prompt

template = """
The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative
and funny responses to the users questions.

User: {query}
AI:
"""

prompt_template = PromptTemplate.from_template(
    template = template
)

prompt = prompt_template.format(
    query = "What is the meaning of life?"
)

print(prompt)

In [ ]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo")
output = llm.invoke(prompt)
print(output.content)

## B- Few-Shot

* `Langchain` provee una clase para poder utilizar este tipo de técnicas: `FewShotPromptTemplate`.
* El objetivo de `FewShotPromptTemplate` es seleccionar ejemplos basados en el input y luego formatear los ejemplos en un prompt final, el cual alimentará el `LLM`.
* Como se observó en ejemplos anteriores se puede usar con `PromptTemplate` o con `MessagePromptTemplate`.

**EJEMPLO 1 con PromptTemplate:**

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate

In [ ]:
# Creamos los ejemplos:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

In [ ]:
# Creamos el template para los ejemplos
example_template = """
User: {query}
AI: {answer}
"""

# Acá es cómo habíamos hecho en los primeros ejemplos
"""
template =
Escribí una función llamada 'imprimir_numeros' en código {language}
que ejecute la siguiente {tarea}.
"""
print()

In [ ]:
print(example_template)

In [ ]:
# Creamos el prompt_template para los ejemplos
example_prompt = PromptTemplate(input_variables = ["query", "answer"], template = example_template)

# Acá es cómo habíamos hecho en los primeros ejemplos
"""
prompt_template = PromptTemplate.from_template(template = template)
"""
print(example_prompt.format(**examples[0]))
print("---" * 20)
print(example_prompt.format(**examples[1]))

In [ ]:
# Agregamos el prompt_template de los ejemplos al prompt_template final, previo a esto agregamos 'prefijo' y 'sufijo':

# Prefijo es como si le dijeramos cómo tiene que ser el comportamiento
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:
"""

# Sufijo es para pasar el formato de input y ouput
suffix = """
User: {query}
AI: """

# Creamos el prompt_template final
prompt_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["query"],
    example_separator = "\n\n"
)

for i in prompt_template:
    print(i)

In [ ]:
prompt  = prompt_template.format(query = "What is the meaning of life?")
print(prompt)

# Acá es cómo habíamos hecho en los primeros ejemplos
"""
prompt = prompt_template.format(language = "python",
                                tarea = "Imprimí los números del 1 al 10")
"""
print()

In [ ]:
llm = OpenAI(model_name = "gpt-3.5-turbo-instruct")

output = llm.invoke(prompt)
print(output)

**EJEMPLO 2 con PromptTemplate:**

In [ ]:
# Creamos los ejemplos:

examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

In [ ]:
# Creamos el template para los ejemplos
example_template = """
Question: {question}
{answer}
"""

# Acá es cómo habíamos hecho en los primeros ejemplos
"""
template =
Escribí una función llamada 'imprimir_numeros' en código {language}
que ejecute la siguiente {tarea}.
"""
print()

In [ ]:
# Creamos el prompt_template para los ejemplos
example_prompt = PromptTemplate(input_variables = ["question", "answer"], template = example_template)

# Acá es cómo habíamos hecho en los primeros ejemplos
"""
prompt_template = PromptTemplate.from_template(template = template)
"""
print(example_prompt.format(**examples[0]))
print("---" * 20)
print(example_prompt.format(**examples[1]))

In [ ]:
# Agregamos el prompt_template de los ejemplos al prompt_template final, previo a esto agregamos 'prefijo' y 'sufijo':

# Prefijo es como si le dijeramos cómo tiene que ser el comportamiento
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:
"""

# Sufijo es para pasar el formato de input y ouput
suffix = """
User: {query}
AI: """

# Creamos el prompt_template final
prompt_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    suffix = "Question: {input}",
    input_variables = ["input"]
)

for i in prompt_template:
    print(i)

In [ ]:
prompt  = prompt_template.format(input = "Who was the father of Mary Ball Washington?")
print(prompt)

# Acá es cómo habíamos hecho en los primeros ejemplos
"""
prompt = prompt_template.format(language = "python",
                                tarea = "Imprimí los números del 1 al 10")
"""
print()

In [ ]:
llm = OpenAI(model_name = "gpt-3.5-turbo-instruct")

output = llm.invoke(prompt)

print(output)

In [ ]:
prompt  = prompt_template.format(input = "Are both the players Lebron James and Cristiano Ronaldo from the same country?")
print(prompt)

# Acá es cómo habíamos hecho en los primeros ejemplos
"""
prompt = prompt_template.format(language = "python",
                                tarea = "Imprimí los números del 1 al 10")
"""
print()

In [ ]:
llm = OpenAI(model_name = "gpt-3.5-turbo-instruct")

output = llm.invoke(prompt)

print(output)

**EJEMPLO 3 con MessagePromptTemplate:**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

In [ ]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

In [ ]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

In [ ]:
chat_template_examples = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

print(chat_template_examples.format())

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        chat_template_examples,
        ("human", "{input}"),
    ]
)
print(f"chat_template:")
for i in chat_template:
    print(i)

In [ ]:
messages = chat_template.format_messages(input = "What is the surface area of a circle whose radius is 2 cm?")
messages

In [ ]:
llm = OpenAI(model_name = "gpt-3.5-turbo-instruct")

output = llm.invoke(messages)

print(output)

## C- Prompt Role

In [ ]:
template = """
You're Carlitos, a friendly and helpful chatbot. Your role is to respond to all comments made to you. Before respond to the user in a sarcastically way, please introduce yourself with a short presentation
Here is the question: {query}
"""

prompt_template = PromptTemplate.from_template(
    template = template
)

prompt = prompt_template.format(
    query = "Why did the chicken cross to the other side?"
)

print(prompt)

In [ ]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo")
output = llm.invoke(prompt)
print(output.content)

## D- Chain of thought

In [ ]:
template = """"
Para la siguiente tarea:
{tarea}
Describe tu razonamiento paso a paso, en bullets.
"""
prompt_template = PromptTemplate.from_template(
    template = template
)

query = """
Un salón de clases tiene dos sillas azules por cada tres sillas rojas.
Si hay un total de 30 sillas en el salón de clases, ¿cuántas sillas azules hay?
"""

prompt = prompt_template.format(
    tarea = query
)

llm = ChatOpenAI(model_name = "gpt-3.5-turbo")#, temperature = 0)
output = llm.invoke(prompt)
print(output.content)

## E- Least to most

In [ ]:
template = """"
Dada la siguiente consulta:
{consulta}

Responde a la siguiente instrucción:
{instruccion}
"""

prompt_template = PromptTemplate.from_template(
    template = template
)

consulta = """
Compré una remera el 1 de marzo. Vi que tenía descuento, así que compré una camisa que originalmente costaba $30 y obtuve un 40% de descuento.
Vi que tenés un nuevo descuento para remeras del 50%.
Me pregunto si puedo devolver la camiseta y tener suficiente crédito en la tienda para comprar dos de tus remeras.
"""

instruccion = """
Sos un agente de servicio al cliente encargado de responder amablemente a las consultas de los clientes.
Se permiten devoluciones dentro de los 30 días. La fecha de hoy es 29 de marzo. Actualmente hay un 50% de descuento en todas las remeras.
Los precios oscilan entre $ 18 y $ 100.
No inventes ninguna información sobre políticas de descuento. ¿Qué subproblemas deben resolverse antes de responder la consulta?
Resuelve cada uno de ellos para dar la respuesta final.
"""

prompt = prompt_template.format(
    consulta = consulta,
    instruccion = instruccion
)

In [ ]:
print(prompt)

In [ ]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo")#, temperature = 0)
output = llm.invoke(prompt)
print(output.content)

# 3- ALGUNAS ESTRATEGIAS ADICIONALES

## A- Seleccionando por longitud

* Selecciona qué ejemplos usar según la longitud.
* Esto es útil cuando nos preocupa crear un mensaje que abarque toda la ventana de contexto.
* Para entradas más largas, seleccionará menos ejemplos para incluir, mientras que para entradas más cortas seleccionará más.

In [ ]:
from langchain_core.example_selectors import LengthBasedExampleSelector
import re

In [ ]:
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [ ]:
example_template = """
Input: {input}
Output: {output}
"""

In [ ]:
example_prompt = PromptTemplate(
    input_variables = ["input", "output"],
    template = example_template,
)

In [ ]:
example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = example_prompt,
    max_length = 25 # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
)
# max_length es el limite maximo en palabras del la variable a formatear + ejemplos.
    # Ejemplo 1: adjective = "big", 25 - 1: tengo 24 palabras para agregar como ejemplo

In [ ]:
prompt_template = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "Give the antonym of every input",
    suffix = "Input: {adjective}\nOutput:",
    input_variables = ["adjective"]
)

In [ ]:
prompt = prompt_template.format(adjective = "big")
print(len(re.split("\n| ", prompt)))
print(prompt)

In [ ]:
llm = OpenAI(model_name = "gpt-3.5-turbo-instruct")
output = llm.invoke(prompt)
print(output)

## B- Seleccionando por similaridad del coseno

In [ ]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

In [ ]:
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [ ]:
examples_template = """
Input: {input}
Output: {output}
"""

In [ ]:
example_prompt = PromptTemplate(
    input_variables = ["input", "output"],
    template = examples_template,
)
for i in example_prompt:
    print(i)

In [ ]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples = examples,
    embeddings = OpenAIEmbeddings(),
    vectorstore_cls = Chroma,
    k = 1,
)
example_selector

In [ ]:
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "Give the antonym of every input",
    suffix = "Input: {adjective}\nOutput:",
    input_variables = ["adjective"],
)

In [ ]:
print(similar_prompt.format(adjective = "worried"))

In [ ]:
print(similar_prompt.format(adjective = "large"))

In [ ]:
prompt = similar_prompt.format(adjective = "large")
llm = OpenAI(model_name = "gpt-3.5-turbo-instruct")
output = llm.invoke(prompt)
print(output)